### k-NN - Linear Algebra with NumPy

$v$ = target vector

$X$ = data points matrix with the vectors as row vectors (not column)

$y$ = the predicted labeled output

Mathematically this implementation can be represented as:

$D = X_m - v^T$ | Difference matrix, subtract vector v from every row (vector) $m \in X$

$||v|| = \sqrt{v \cdot v}$

Normally, to get the length of every vector $m$ in matrix $X$, we can do $P = A^TA, P_{n,n} = m \cdot m $ but in this case the matrix contains row vectors, therefore $P = AA^T$ is correct. The squared length will be at every pivot position in the matrix.

$P = AA^T$

$l_n \in P : l_n = \sqrt{P_{n,n}}$

Now we have the length of all vectors $v \in D$ the rest will be done by using argsort and take the majority vote with np functions. 


In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('iris_train.csv')
df_test = pd.read_csv('iris_test.csv')
targets = ['Iris-virginica', 'Iris-setosa', 'Iris-versicolor']

print(df_test.values[40, :])

for i in range (len(targets)):
    df.loc[df['class'] == targets[i], 'class'] = i
    df_test.loc[df_test['class'] == targets[i], 'class'] = i

data = df.values[:,:4]
y = df.values[:,4:]
point = df_test.values[40, :4]


FileNotFoundError: [Errno 2] No such file or directory: 'iris_train.csv'

In [2]:
def knn_linear_algebra(n, X, y, v):
    global targets
    
    difference_vectors = X - v
    target_vector_length = (v @ v) ** (1/2)
    print(target_vector_length)
    
    #Length of vectors n will be in (n,n), i.e in the pivots/diagonal
    vector_length_matrix = (difference_vectors @ np.transpose(difference_vectors)) ** (1 / 2)
    
    vector_length_list = np.diagonal(vector_length_matrix)
    sorted_length_indices = np.argsort(vector_length_list)
    closest_vectors = sorted_length_indices[:n]
    
    majority_list = list(y[closest_vectors, 0])
    prediction = np.argmax(np.bincount(majority_list))
    
    return targets[prediction]


knn_linear_algebra(10, data, y, point)
        

NameError: name 'data' is not defined